# LAB: Quantum Key Distribution Lab

Now that you know how to operate the quED, I want you to take your knowledge of QKD and generate a key via BB84. For this write-up, I want you to focus on building a model of how the BB84 will work and then comparing your experiment to that model. 

## Goal of this lab

In this lab, you'll be generating a secret key via BB84 quantum key distribution (QKD). This will involve several steps:
1. Model the secret key rate you expect to see
1. Verify you have a single photon source based on your measurements from the HBT.
1. Generate a raw key and compare it to your model
1. Perform post processing to generate a final key
1. Propose next steps to improve the experiment

## Model the secret key rate you expect to see
**Note to Jane: figure out how to make the ket notations work in Latex**



* How fast can Alice encode photons?
* What's the loss between Alice's source and Bob's measurement? If Alice sends 1 photon, what's the probability Bob gets a click?
    Jane will tell you the transmission through the polarization optics and an estimate of the APD efficiency
* How much of the raw key gets lost through sifting, BER estimate, information reconciliation, and privacy amplification?
 

## Verify mean number of photons

### BB84's security is based on the no-cloning theorem
The no-cloning theorem states that for a given quanta in a superposition of eigenstates, it's impossible to produce a perfect copy. See, eg [Wootters and Zurek 1982]("https://www.nature.com/articles/299802a0") or [Dieks 1982]("https://www.sciencedirect.com/science/article/pii/0375960182900846?via%3Dihub").

Photon number splitting attack [Brassard, Lutkenhaus, Mor, Sanders 2000]("https://doi.org/10.1103/PhysRevLett.85.1330")

As you recall from the reading, the HBT experiment can be used to verify that we have a heralded single-photon source. Use your results from the last lab to justify why the quED is an appropriate source of single photons.



## Establish correct polarization

Unless you want to rotate the half wave plate (HWP) and polarization optics manually for every single bit, you're going to want to make sure you can control polarization optics via a Python script. We've provided a basic script for you below, which you can edit as you see fit.

Before you test the script, you need to make sure that your HWP and polarizers are lined up where you think they are, ie if you send $0^{\circ}$ to the HWP, how do you know if it's actually at $0^{\circ}$? For the quTools set up, you can set the angle and then manually rotate the polarization to get the optics where you want them. Ask one of the instruction staff to show you how to do that. 

## Generate bits and bases
1. Start with many photons per bit
1. Generate all the bases at the beginning

In [47]:
# Import necessary Libraries

# Generate the basis for each bit from Alice and Bob

import numpy as np
import random
import matplotlib.pyplot as plt

LENGTH = 150;

# Are we using PBS and 2 detectors?
PBS  = False

# Assign bit values
# How can I actually incorporate this usefully?
bH = 0
bV = 1
bD = 0
bA = 1

# Initialize the random bases for Alice and Bob. Let 0 be HV and 1 be AD
basis_Alice = np.zeros((LENGTH))
basis_Bob = np.zeros((LENGTH))

# Now initialize the bit values for Alice
bits_Alice = np.zeros((LENGTH))
bits_Bob = np.zeros((LENGTH)) 

# Now set the correct values for Alice and Bob's rotation
angles_Alice = np.zeros((LENGTH))
angles_Bob = np.zeros((LENGTH))
for x in range (LENGTH):
    basis_Alice[x] = random.randint(0,1)
    basis_Bob[x] = random.randint(0,1)
    bits_Alice[x] = random.randint(0,1)
    bits_Bob[x] = random.randint(0,1)
    angles_Alice[x] = 22.5*basis_Alice[x] - 45*bits_Alice[x]
    if PBS:
        angles_Bob[x] = 22.5*basis_Bob[x] # this will rotate Bob's HWP as needed. You'll measure from 2 detectors
    else:
        angles_Bob[x] = 45*basis_Bob[x] - 90*bits_Bob[x] # this will tells us how to rotate the linear polarizer

# print('Alices Basis :',basis_Alice)
# print('Alices Bits  :',bits_Alice)
# print("Alices Angles: %s" %(angles_Alice))
# print('Bob basis: ', basis_Bob)
# print('Bob bits:  ',bits_Bob)
# print('Bob angles:',angles_Bob)




In [48]:
# Place holder for actual data gathering
TT = 1.0 # set a value for transmission through system, loss, etc.

detH = []
detV = []

for x in range(LENGTH):
    angle = angles_Alice[x]
    if random.random() <= TT:
        if PBS:
            if angle == 0 or abs(angle) == 45:
                if angles_Bob[x] == 0:
                    detH.append(int(angle == 0))
                    detV.append(int(angle == 45))
            else: # Alice is in D or A basis
                if angles_Bob[x] == 22.5: # Bob is also in D or A basis
                    detH.append(int(angle < 0))
                    detV.append(int(angle > 0))
            Hval = random.randint(0,1)
            detH.append(Hval)
            Vval = 1 - Hval
            detV.append(1 - Hval)
        else:
            if angle*2 == angles_Bob[x]:
                detH.append(1)
            else:
                detH.append(0)
            detV.append(-1)
    else:
        detH.append(-1)
        detV.append(-1) 
# print(detH)
# print(detV)
# print(bits_Alice ==)

## Sift the raw bits

You need to check several things:
   1. Did Bob get a click?
   1. If yes, did he have the same basis as Alice?
   
Based on your pre-lab, how many bits did you expect to see in the key (after sifting and comparing bases)? Why do you think the experiment differs from that expectation?

In [49]:
Bob_all = []
Al_all = []


detH = np.array(detH)
detV = np.array(detV)
# find cases where Bob gets exactly 1 click
mask = (detH == 1) ^ (detV == 1) 
# returns array indices where Bob gets exactly 1 click
ix = np.array(np.where(mask))[0] 
# find raw key length
rkl = ix.size 

# look for cases where Alice and Bob have the same basis
if PBS:
    print('Jane needs to write this code')
else:
    for x in range(rkl):
        nn = ix[x]
        AA = angles_Alice[nn]
        AB = angles_Bob[nn]
        if AA*2 == AB:
            Al_all.append(bits_Alice[nn])
            Bob_all.append(bits_Bob[nn])


print(Al_all == Bob_all)
# print(Bob_all)

True


## Information Reconciliation/Error Correction

Alice and Bob want to make sure that any errors in their key are corrected, but they don't want Eve to get too much information. The figure of merit for information reconiciliation/error correction is bit error rate (BER), the ratio of bits that are incorrect.

A basic error correction scheme is provided below. It first estimates the raw BER by having Alice and Bob compare the first `length_reveal` bits over a classical channel. Eve gets all the information Alice and Bob share over the classical channel, so Alice and Bob throw away those bits as they are no longer secret.

**Write-up question:** Where do the errors come from? How is that reflected in your model? Think about the limitations of the polarization optics and dark counts, as a place to start.

Another form of error correction is available here: I found a description of Cascade Error Correction [here](https://cascade-python.readthedocs.io/en/latest/protocol.html).

In [20]:
# Estimate the q BER by revealing some of the bits.

length_reveal = 100

# errors = (bits_tx_sifted[0:length_reveal - 1] != bits_rx_sifted[0:length_reveal - 1]).astype(int)
# errors_total = errors.sum()
# BER = 1.0/length_reveal * errors.sum()

Al_all = np.random.randint(low = 0, high = 2, size = (length_reveal+30,))

# give Bob some errors for testing
err_vec = np.random.choice(2,length_reveal+30,p=[0.9,0.1])
Bob_all = Al_all ^ err_vec

key_test = Al_all[0:length_reveal]
Bob_test = Bob_all[0:length_reveal]
BERest =  np.mean(key_test != Bob_test)
print(BERest)

# Discard those bits
Al_all = Al_all[length_reveal:]
Bob_all = Bob_all[length_reveal:]





0.09


Now that Alice and Bob have an estimate of the quantum BER, they can evaluate to what degree Eve is interfering in their key distribution and if error correction coding will help them. If the qBER is too high (for this particular scheme, the cutoff is when qBER $\geq 0.14$), the error-correction code will actually make the BER worse. This scheme can correct 1 error in every block of 7 bits.

The below section uses a very simple one way Hamming code following the example in the section "Error Correction for Quantum Key Distribution" in Loepp and Wootters [book available online with MIT Library sign in](https://www.cambridge.org/core/books/abs/protecting-information/quantum-cryptography-revisited/8A88AA6C8C88E8A07E535FACEDE3C310). This allows Alice and Bob to reveal a minimum amount of information to Eve while doing error correction; the privacy amplification step then removes the information they did reveal to Eve.

The Hamming code works in the following way: Bob and Alice take their raw key (`a` for Alice and `b` for Bob) and break it into blocks of 7 bits. Each block is multiplied by a 3x7 Hamming matrix `H`. For Alice, this results in `Ha` and for Bob `Hb`. Then Alice sends `Ha` to Bob. Bob finds `Ha - Hb = He` where `e` is the error vector, ie the vector that indicates whether `a == b` for each element. From there, Bob finds `e` and can use that to locate and then correct his bits. For a more in-depth explanation, I highly recommend checking out the linked section.

In [14]:
# Working out the Hamming Error matrix once

n = 7
k = np.log2(n+1).astype(int)

H = np.array([[1,1,1,0,1,0,0],
              [1,1,0,1,0,1,0],
              [0,1,1,1,0,0,1]])

# Define the error correction functions we'll need

def errCor(err_mat,H,a,b,n):
    # Multiply Alice's bits a by H
    at = np.matrix.transpose(a)
    Hat = np.matmul(H,at)
    Hat = np.remainder(Hat,2) # we're doing bitwise operation, so we need to eliminate the overflow

    # Multiply Bob's bits b by H
    bt = np.matrix.transpose(b)
    Hbt = np.matmul(H,bt)
    Hbt = np.remainder(Hbt,2)
    
    # Find Hb - Ha = s 
    s = Hat ^ Hbt
    sT = np.matrix.transpose(s)
    
    # s = He where e is the error vector.
    # If we know s, we can find which row of the error matrix it matches
    # Based on that, we know where the error is
    x = np.where((err_mat==sT).all(axis=1))[0][0].astype(int)
    err_vec = np.zeros((1,n))

    if x < n: 
     err_vec[0,x] = 1
     
    ev = err_vec.astype(int)
    ev = np.reshape(ev,(1,n))

    
    # Bob now has a corrected vector
    # bc = binSub(b,err_vec)
    bc = b ^ ev
    return bc

def makeErrMat(n,k,H):
    err_mat = np.zeros((n+1,k))

    for ii in range(n):
        err_vec = np.zeros((n,1))
        err_vec[ii] = 1
        HeT = np.matmul(H,err_vec)
        HeTT = np.matrix.transpose(HeT)
        # print('err loc',err_vec)
        # print('HeT', HeT)
        err_mat[ii] = HeTT    

    err_mat = err_mat.astype(int)
    return err_mat

err_mat = makeErrMat(n,k,H)

In [16]:
# do error correction for each block

# The following is an example script
LL = 31

Al_all = np.random.randint(low = 0, high = 2, size = (LL,))



err_all = np.zeros((LL)).astype(int)
err_all[3] = 1
err_all[26] = 1
err_all[18] = 1


Bob_all = Al_all ^ err_all


# We've skipped sifting for now
sk = Bob_all.size

cutoff =  sk % n # find key length modulo n

# elimate the "cut-off" bits
Bob = Bob_all[0:-cutoff]

# reshape Alice and Bob's bits into a 2d matrix
a = np.array([[1,0,0,0,1,1,1]])
b = np.array([[1,0,0,0,1,1,0]])


Bob = np.reshape(Bob,(-1,n))
Al  = np.reshape(Al_all[0:-cutoff],(-1,n))
Al[1,:] = a
Bob[1,:] = b

shape = np.shape(Al)
Bob_corr = np.zeros((shape)).astype(int) #initialize matrix for Bob's error correction
words = shape[0]

for ii in range(words):
    Al_word = Al[ii,:]
    Bob_word = Bob[ii,:]
    Bob_word_corr = errCor(err_mat, H, Al_word,Bob_word,n)
    Bob_corr[ii,:] = Bob_word_corr
    
error = np.mean(Al != Bob_corr)
print(error)


0.0


## Privacy Amplification

In this set-up, Eve isn't interacting with the optical set-up, so the only information she has is what Alice and Bob share via the classical channel. 
So, following pg 184 from the above book, we simply get rid of the last 3 bits of each of the 7-word chunks, and Eve will have no information about Alice and Bob's shared key.

There are a variety of ways to perform error correction and privacy amplification. The privacy ammplification needs to take into account how the error correction is done: if Alice and Bob reveal more information (which would allow the error correction to work for a worse qBER) they need to eliminate more bits to thwart Eve.

In [ ]:
# Take the giant matrix of error-correct keys, and just skip the last few columns (or rows)
Bob_corr = Bob_corr[:,0:4]
Al = Al[:,0:4]

# Then reshape the key into one long string of bits
Bob_key = np.reshape(Bob_corr,(1,-1))
Al_key = np.reshape(Al,(1,-1))

## Conclusion
In your write-up, think about one way you could improved the SKR using commerically available components. How practical is the component for the speed-up? 